# Домашнее задание 4

В этом задании мы:
1. Построим классификатор датасета CIFAR с помощью обычных нейросетей и CNN.
2. Поработаем с аугментациями и добьемся большего качества с их помощью.
3. Попрактикуемся с техникой fine-tuning: возьмем готовый MobileNet и дообучим последний слой под нашу задачу.

## Классификация: CNN против обычных сетей

В первой части задания мы повторим то, что проделывалось на лекции.
Но на этот раз будем учить сети до победного, пока их качество не перестанет улучшаться - и сравним результаты.

Также в конце оценим число параметров в каждой сети, чтобы сравнить эффективность CNN и FC при работе с изображениями.

Воспользуемся датасетом CIFAR.

In [1]:
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor

train_dataset = CIFAR10(root="./data", train=True, download=True, transform=ToTensor())
test_dataset = CIFAR10(root="./data", train=False, download=True, transform=ToTensor())

100%|██████████| 170498071/170498071 [00:50<00:00, 3398920.74it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


### Задание №1

Создайте два объекта `DataLoader` и сохраните их в переменные `train_loader` и `test_loader` (для тренировочной и тестовой выборки соответственно).

Используйте размер батча 256.

In [2]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

### Задание №2

Обучите полносвязную сеть для классификации CIFAR.

Достаточно 3 блоков "Linear + ReLU".
Ваша задача - вывести accuracy на _тестовой выборке_ на плато.
Т.е. нужно обучить сеть настолько долго, чтобы увидеть, как ее качество перестает расти с ростом числа эпох.
Для этого попробуйте подвигать `lr` и `num_epochs`.


Сдайте в ЛМС предельный accuracy, который может достичь полносвязная сеть.

In [62]:
from torch import optim
import matplotlib.pyplot as plt
from IPython.display import clear_output


@dataclass
class TrainConfig:
    lr: float
    num_epochs: int


def plot_accuracy(epoch: int, values: list[float]):
    """Пример:

    >>> acc.append(validation_accuracy)
    >>> plot(i + 1, validation_accuracy)
    """
    clear_output(True)
    plt.title("Epoch %s. Accuracy: %s" % (epoch, values[-1]))
    plt.plot(values)
    plt.grid()
    plt.show()


import matplotlib.pyplot as plt
from IPython.display import clear_output


@dataclass
class TrainConfig:
    lr: float
    num_epochs: int


def plot_accuracy(epoch: int, values: list[float]):
    """Пример:

    >>> acc.append(validation_accuracy)
    >>> plot(i + 1, validation_accuracy)
    """
    clear_output(True)
    plt.title("Epoch %s. Accuracy: %s" % (epoch, values[-1]))
    plt.plot(values)
    plt.grid()
    plt.show()


def train_loop(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    optimizer: torch.optim.Optimizer,
    config: TrainConfig,
    device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu"),
):
    # Перенос модели на GPU, если доступен
    model = model.to(device)
    val_acc = []
    # Цикл по эпохам
    for i in range(config.num_epochs):
        model.train()  # Переключение в режим обучения

        for X_batch, y_batch in train_loader:
            print(X_batch)
            # Перенос батча на GPU
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            # Обнуление градиентов
            optimizer.zero_grad()

            # Получение предсказаний модели
            outputs = model(X_batch)

            # Расчёт потерь
            loss = F.cross_entropy(outputs, y_batch)
            loss.backward()  # Обратное распространение ошибки

            # Обновление параметров модели
            optimizer.step()

        # Валидация
        model.eval()  # Переключение в режим валидации
        total_val_acc = 0
        total_val_samples = 0

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                # Перенос батча на GPU
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)

                # Получение предсказаний модели
                val_outputs = model(X_batch)

                # Накопление статистики
                total_val_acc += (val_outputs.argmax(1) == y_batch).sum().item()
                total_val_samples += X_batch.size(0)

        # Расчёт средней потери и точности на валидационном наборе
        val_acc.append(total_val_acc / total_val_samples)
        plot_accuracy(i + 1, val_acc)

    return val_acc

# class SimpleFCNModel(nn.Module):
#     def __init__(self, num_classes=10):
#         super().__init__()
#         self.fc = nn.Sequential(
#             nn.Flatten(),
#             nn.Linear(3 * 32 * 32, 512),
#             nn.ReLU(),
#             nn.Linear(512, 256),
#             nn.ReLU(),
#             nn.Linear(256, 128),
#             nn.ReLU(),
#             nn.Linear(128, num_classes),
#         )
#
#     def forward(self, x):
#         return self.fc(x)
#
#
# torch.manual_seed(987)
# params = TrainConfig(lr=1e-3, num_epochs=50)
# model = SimpleFCNModel()
# optimizer = torch.optim.Adam(model.parameters(), lr=params.lr)
# train_loop(model, train_loader, test_loader, optimizer, params)
# Около 0.55 должно выйти

In [6]:
def report_parameters(model: nn.Module):
    print(
        "Суммарное количество параметров:",
        sum(p.nelement() for p in model.parameters()),
    )
    print(
        "Суммарный размер (Мб) параметров:",
        sum(
            parameter.nelement() * parameter.element_size()
            for parameter in model.parameters()
        )
        / 1024**2,
    )

In [18]:
report_parameters(model)

Суммарное количество параметров: 1738890
Суммарный размер (Мб) параметров: 6.633338928222656


### Задание №3

Теперь постройте и обучите CNN сеть.
Опять же, не используйте глубокую сеть: мы хотим иметь схожее количество параметров для сравнения.

Достаточно будет трех блоков "Conv + ReLU + MaxPool".

In [31]:
class SimpleCNNModel(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv_net = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc = nn.Linear(128 * 4 * 4, num_classes)

    def forward(self, x):
        x = self.conv_net(x)
        x = x.view(-1, 128 * 4 * 4)
        return self.fc(x)


torch.manual_seed(987)
model = SimpleCNNModel()

params = TrainConfig(lr=1e-2, num_epochs=50)
optimizer = torch.optim.Adam(model.parameters(), lr=params.lr)
train_loop(model, train_loader, test_loader, optimizer, params)

[0.492,
 0.5322,
 0.5636,
 0.5883,
 0.6066,
 0.6271,
 0.6192,
 0.6468,
 0.6407,
 0.6423,
 0.6321,
 0.6529,
 0.6519,
 0.6542,
 0.6479,
 0.6538,
 0.6485,
 0.6675,
 0.6694,
 0.6625,
 0.6511,
 0.6547,
 0.6494,
 0.6686,
 0.6509,
 0.6604,
 0.6368,
 0.6554,
 0.665,
 0.6739,
 0.6696,
 0.6693,
 0.6466,
 0.6279,
 0.6543,
 0.6418,
 0.6591,
 0.6662,
 0.6654,
 0.662,
 0.6451,
 0.6628,
 0.6554,
 0.665,
 0.6648,
 0.6586,
 0.6715,
 0.6528,
 0.6745,
 0.6697]

In [19]:
report_parameters(model)

Суммарное количество параметров: 156074
Суммарный размер (Мб) параметров: 0.5953750610351562


Обратите внимание на качество и на число параметров.
Качество получается выше, а число параметров - на порядок меньше.

Делаем вывод, что CNN позволяют выбивать лучшее качество, чем обычные сети, и при меньшем числе параметров.

Но CNN - не единственный способ улучшить качество при работе с картинками.

### Задание №4
Реализуйте следующие аугментации:
1. Горизонтальное отражение (Horizontal Flip) с вероятностью применения 30%
2. Вращение на угол (Rotate), близкий к 30 градусам, с вероятностью применения около 30%.
3. Random Resized Crop - тут выберите нужные параметры самостоятельно.
4. Normalize. Нормализовать нужно вдоль трех осей изображения. Среднее и std подсчитайте самостоятельно, используя `train_dataset` (в подсчет статистик _нельзя_ включать `test_dataset`).

Используйте библиотеку `albumentations`.
Не забудьте, что `albumentations` работает с numpy-массивами.
Придется перегонять данные из pytorch в numpy-массивы и обратно:

```python
np_array = tensor.numpy()
tensor_back = torch.from_numpy(np_array)
```

Сохраните аугментации в переменную `transforms` и сдайте свой код в ЛМС.

<details>
<summary>Как ваш код будет проверяться</summary>

```python
import albumentations as A

# <Ваш код здесь>

# Затем проверки на переменную transforms
assert some_check(transforms)
assert another_check(transforms)
```
</details>

In [36]:
!pip install albumentations==1.4.2


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

image_size = 32

transforms = A.Compose(
    [
        A.HorizontalFlip(p=0.3),
        A.Rotate(limit=30, p=0.3),
        A.RandomResizedCrop((image_size, image_size), scale=(0.8, 1.0), p=0.3),
        A.Normalize(
            mean=[0.4914, 0.4822, 0.4465],
            std=[0.24703279, 0.24348423, 0.26158753],
            max_pixel_value=1.0,
        ),
        ToTensorV2(),
    ]
)

## Аугментации
Зачастую аугментации помогают увеличить качество модели.
Объясняется это так: аугментация изображений обогащает датасет новыми картинками, сгенерированными из существующих.
Переобучения не происходит, потому что мы не просто дублируем изображения, а немного изменяем их.
### Задание №5

Обучите CNN с использованием аугментаций.
Как и в прошлых заданиях, держите обучение до конца - пока loss не выйдет на плато.

Ваша задача - получить accuracy выше 76%.
Сдайте в ЛМС:
- код класса модели. Класс должен называться `SimpleCNNModel`;
- .pt файл с обученной моделью;

In [80]:
from torch.utils.data import Dataset


class DatasetWithTransforms(Dataset):
    def __init__(self, original_dataset: Dataset, transforms: A.Compose):
        super().__init__()
        self._transforms = transforms
        self._dataset = original_dataset

    def __len__(self):
        return len(self._dataset)

    def __getitem__(self, index: int):
        print('###', self._dataset)
        img, label = self._dataset[index]
        img_as_np = img.numpy()
        # albumentations принимает картинку в формате (w, h, c), но pytorch хранит в (c, w, h)
        # np_array.transpose(1, 2, 0) сделает так, чтобы сначала шла ось 1, потом ось 2, потом ось 0
        # т.е. (c, w, h) перейдет в (w, h, c)
        #       0  1  2              1  2  0
        transformed = self._transforms(image=img_as_np.transpose(1, 2, 0))["image"]
        print('@@@', type(img_as_np))
        return transformed, label


train_dataset_augs = DatasetWithTransforms(train_dataset, transforms)
train_loader_augs = DataLoader(
    train_dataset_augs, batch_size=256, shuffle=True, num_workers=0
)

# Надо не забыть в тесте нормировать картинки на те же статистики, что в train датасете
transforms_test = A.Compose(
    [
        A.Normalize(
            mean=[0.4914, 0.4822, 0.4465],
            std=[0.24703279, 0.24348423, 0.26158753],
            max_pixel_value=1.0,
        ),
        ToTensorV2(),
    ]
)
test_dataset_augs = DatasetWithTransforms(test_dataset, transforms_test)
test_loader_augs = DataLoader(
    test_dataset_augs, batch_size=256, shuffle=False, num_workers=0
)

torch.manual_seed(seed=987)
model = SimpleCNNModel()

params = TrainConfig(lr=1e-3, num_epochs=50)
optimizer = torch.optim.Adam(model.parameters(), lr=params.lr)
x, y = next(iter(train_dataset))
print(x)
train_loop(model, train_loader_augs, test_loader_augs, optimizer, params)
torch.save(model.state_dict(), "model.pt")

TypeError: image must be numpy array type

Аугментации улучшили качество.

Заметьте, что нормализацию можно было бы применить и в прошлом пункте, чтобы более честно оценить, какой прирост дали развороты и вращения изображения.
Советуем самостоятельно провести эксперимент и увидеть различия.

<details>
    <summary>Какие результаты ожидать</summary>
    У авторов получилось около 73% accuracy при использовании только лишь нормализации. При добавлении остальных аугментаций качество было еще выше.
</details>

## Transfer learning
### Задание №6
Transfer learning состоит в том, чтобы взять готовую сеть и дообучить небольшую ее часть.
В этом задании мы будем учить FC слой в конце MobileNet.

Загрузите предварительно обученную модель из серии `MobileNet`, используйте `MobileNet_V3_large`.

Поменяйте ее последний слой (классификатор) на один линейный слой.
Обучите все это дело, меняя **только** параметры своего слоя (подумайте, что передавать в оптимизатор).
Сохраните обученный слой (и только его) в `model_finetune.pt`.

Сдайте в ЛМС .pt файл и код, создающий вашу модель в переменную `model_finetune`.
Чтобы сдать это задание, достаточно набрать accuracy > 40%.

In [83]:
from tqdm import tqdm
from torchvision import models, transforms

model = models.mobilenet_v3_large(pretrained=True)
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 10)

# Заморозка всех слоев, кроме последнего
for param in model.parameters():
    param.requires_grad = False
for param in model.classifier[-1].parameters():
    param.requires_grad = True

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # MobileNet ожидает изображения 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

# Определение оптимизатора (только для последнего слоя)
optimizer = optim.Adam(model.classifier[-1].parameters(), lr=1e-3)

# Определение функции потерь
criterion = nn.CrossEntropyLoss()

# Обучение модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader):.4f}")

# Валидация модели
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on test set: {accuracy:.2f}%")

# Сохранение только последнего слоя
torch.save(model.classifier[-1].state_dict(), "model_finetune.pt")

# Переменная для сдачи в LMS
model_finetune = model.classifier[-1]

Files already downloaded and verified
Files already downloaded and verified


Epoch 1/5: 100%|██████████| 782/782 [00:51<00:00, 15.24it/s]


Epoch 1, Loss: 0.7000


Epoch 2/5: 100%|██████████| 782/782 [00:50<00:00, 15.44it/s]


Epoch 2, Loss: 0.5500


...

Epoch 3, Loss: 0.5251


Epoch 4/5: 100%|██████████| 782/782 [00:52<00:00, 14.80it/s]


Epoch 4, Loss: 0.5168


Epoch 5/5: 100%|██████████| 782/782 [00:51<00:00, 15.08it/s]


Epoch 5, Loss: 0.5079
Accuracy on test set: 82.27%


In [ ]:
print(
    "Суммарное количество параметров:",
    sum(p.nelement() for p in trainable_params),
)
print(
    "Суммарный размер (Мб) параметров:",
    sum(
        parameter.nelement() * parameter.element_size()
        for parameter in trainable_params
    )
    / 1024**2,
)

Качество, возможно, просело, зато учим намного меньше параметров.

## Задание №7
Возьмите предпоследний слой вашей CNN модели (тот, что до классификатора).
Этот слой выдает вектора.

Возьмите любой объект из класса 0, подсчитайте его косинусную схождесть со всеми остальными объектами из класса 0, усредните.
Затем подсчитайте то же число, только против всех объектов из класса 1, тоже усредните.
Отправьте в ЛМС два числа, разделенные запятой. Например, "1, 1".

In [ ]:
...

#### Небольшой бонус
Эмбеддинги можно визуализировать, используя t-SNE.
Посмотрите, что получается, попробуйте объяснить картину.

Если модель достаточно качественная, то схожие классы должны собираться в одну кучку, при этом у кучек должны прослеживаться границы.
Конечно же, не забывайте про выбросы.

In [ ]:
from sklearn.manifold import TSNE

result = torch.empty((0, 2048))
labels = []
with torch.no_grad():
    embedding_model.cpu()
    for x_batch, y_batch in test_loader_augs:
        embedding = embedding_model(x_batch).flatten(1)
        embedding /= embedding.norm()
        result = torch.concat((result, embedding))
        labels.extend(y_batch.tolist())

tsne = TSNE(random_state=42)
plot_data = tsne.fit_transform(result.numpy())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 9))
scatter = ax.scatter(
    plot_data[:, 0],
    plot_data[:, 1],
    c=labels,
    cmap="viridis",
    edgecolor="k",
    s=20,
    alpha=1,
)
plt.colorbar(scatter)